<a href="https://colab.research.google.com/github/Sanchitraina1999/LP-I/blob/master/HPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-* 
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge 
!apt-get remove cuda-* 
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-12-20 04:26:18--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-12-20 04:26:18--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?AK3ON5pFjkIVjkjdF8uYXnIxxeLCySaJdwvsMn2kzOpsmc-WZd6jmcX98mFFPMcW3xBXnVo0gs_eeqgjvCzXcddwmX00el37hdDqY1XCH5omP

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-rliddquv
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-rliddquv
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=492f495acdfa3da17167b97d2439138f271f7bbd333427a2de6c07f97b65f788
  Stored in directory: /tmp/pip-ephem-wheel-cache-8pqooiut/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [ ]:
%reload_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


A1: **a) Implement Parallel Reduction using Min, Max, Sum and Average
operations.
b) Write a CUDA program that, given an N-element vector, find-
• The maximum element in the vector
• The minimum element in the vector
• The arithmetic mean of the vector
• The standard deviation of the values in the vector
Test for input N and generate a randomized vector V of length N (N should be large). The program should generate output as the two computed maximum values as well as the time taken to find each value.**

In [ ]:
%%cu
#include "bits/stdc++.h"

using namespace std;

#define N 8
size_t bytes;

__global__ void min(double * array, int n) {
  unsigned int id = blockDim.x * blockIdx.x + threadIdx.x;
  int step_size = 1;

  while (n > 0) {
    if (id < n) {
      int i = (int) id * step_size * 2;
      int j = i + step_size;
      if (array[i] > array[j])
        array[i] = array[j];
    }
    step_size = step_size * 2;
    n = n / 2;
  }
}

__global__ void max(double * array, int n) {
  unsigned int id = blockDim.x * blockIdx.x + threadIdx.x;
  int step_size = 1;

  while (n > 0) {
    if (id < n) {
      int i = (int) id * step_size * 2;
      int j = i + step_size;
      if (array[i] < array[j])
        array[i] = array[j];
    }
    step_size = step_size * 2;
    n = n / 2;
  }
}

__global__ void sum(double * array, int n) {
  unsigned int id = blockDim.x * blockIdx.x + threadIdx.x;
  int step_size = 1;

  while (n > 0) {
    if (id < n) {
      int i = (int) id * step_size * 2;
      int j = i + step_size;
      array[i] = array[i] + array[j];
    }
    step_size = step_size * 2;
    n = n / 2;
  }
}

__global__ void mean_diff_square(double * array, double avg, int n) {
  unsigned int id = blockDim.x * blockIdx.x + threadIdx.x;
  if (id < n)
    array[id] = (array[id] - avg) * (array[id] - avg);
}

int main() {
  double * array = new double[N];
  cout << "array: ";
  for (int i = 0; i < N; i++) {
    array[i] = rand() % 97;
    cout << array[i] << " ";
  }
  cout << endl;

  double result;
  double * d_array;
  bytes = N * sizeof(double); //calculate no. of bytes for array
  cudaMalloc( & d_array, bytes);

  //SUM
  cudaMemcpy(d_array, array, bytes, cudaMemcpyHostToDevice);
  sum << < 1, N / 2 >>> (d_array, N / 2);
  cudaDeviceSynchronize();
  cudaMemcpy( & result, d_array, sizeof(double), cudaMemcpyDeviceToHost);
  cout << "sum: " << result << endl;

  //AVG
  double avg = result / N;
  cout << "avg: " << avg << endl;

  //MIN
  cudaMemcpy(d_array, array, bytes, cudaMemcpyHostToDevice);
  min << < 1, N / 2 >>> (d_array, N / 2);
  cudaDeviceSynchronize();
  cudaMemcpy( & result, d_array, sizeof(double), cudaMemcpyDeviceToHost);
  cout << "min: " << result << endl;

  //MAX
  cudaMemcpy(d_array, array, bytes, cudaMemcpyHostToDevice);
  max << < 1, N / 2 >>> (d_array, N / 2);
  cudaDeviceSynchronize();
  cudaMemcpy( & result, d_array, sizeof(double), cudaMemcpyDeviceToHost);
  cout << "max: " << result << endl;

  //STD
  cudaMemcpy(d_array, array, bytes, cudaMemcpyHostToDevice);
  mean_diff_square << < 1, N >>> (d_array, avg, N);
  cudaDeviceSynchronize();
  sum << < 1, N / 2 >>> (d_array, N / 2);
  cudaDeviceSynchronize();
  cudaMemcpy( & result, d_array, sizeof(double), cudaMemcpyDeviceToHost);
  double variance = result / N;
  cout << "std: " << sqrt(variance) << endl;

  cudaFree(d_array);
  return 0;
}

array: 46 24 91 22 91 8 80 12 
sum: 2.12203e-314
avg: 2.65254e-315
min: 2.12203e-314
max: 2.12203e-314
std: 5.15028e-158



A2:  **Vector and Matrix Operations-
Design parallel algorithm to
Add two large vectors
Multiply Vector and Matrix
Multiply two N × N arrays using n2 processors**


In [ ]:
%%cu
#include "bits/stdc++.h"
using namespace std;

#define N 8
size_t bytes;

__global__ void vector_addition(int *a, int *b, int *c, int n){
    unsigned int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if(idx < n)
        c[idx] = a[idx] + b[idx];
}

int main() {
    int *a = new int[N];
    cout<<"a: ";
    for(int i=0; i<N; i++){
        a[i] = rand()%97;
        cout<<a[i]<<" ";
    }
    cout<<endl;

    int *b = new int[N];
    cout<<"b: ";
    for(int i=0; i<N; i++){
        b[i] = rand()%97;
        cout<<b[i]<<" ";
    }
    cout<<endl;

    bytes = N*sizeof(int);

    int *d_a;
    cudaMalloc(&d_a, bytes);
    cudaMemcpy(d_a, a, bytes, cudaMemcpyHostToDevice);

    int *d_b;
    cudaMalloc(&d_b, bytes);
    cudaMemcpy(d_b, b, bytes, cudaMemcpyHostToDevice);

    int *c = new int[N]{0};
    int *d_c;
    cudaMalloc(&d_c, bytes);

    int threads = 64;
    int blocks = ceil(float(N)/float(threads));

    vector_addition<<<blocks,threads>>>(d_a, d_b, d_c, N);
    cudaDeviceSynchronize();
    cudaMemcpy(c, d_c, bytes, cudaMemcpyDeviceToHost);

    cout<<"c: ";
    for(int i=0; i<N; i++) cout<<c[i]<<" ";
    cout<<endl;

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    return 0;
}

a: 46 24 91 22 91 8 80 12 
b: 17 53 50 58 85 17 23 70 
c: 0 0 0 0 0 0 0 0 



In [ ]:
%%cu
#include "bits/stdc++.h"
using namespace std;

#define N 8
#define M 4
size_t bytes;

// 1xN * NxM = 1xM
__global__ void vecmat_multiplication(int* vector, int* matrix, int* result, int n, int m){
    unsigned int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if(idx < m){
        int sum = 0;
        for(int i=0; i<n; i++)
            sum += vector[i]*matrix[i*m+idx];

        result[idx] = sum;
    }
}

void print_grid(int* array, int n, int m){
    for(int i=0; i<n; i++){
        for(int j=0; j<m; j++)
            cout<<array[i*m+j]<<" ";
        cout<<endl;
    }
    cout<<endl;
}

int main() {
    int *vector = new int[N];
    for(int i=0; i<N; i++)
        vector[i] = rand()%97;

    int *matrix = new int[N*M];
    for(int i=0; i<N*M; i++)
        matrix[i] = rand()%97;

    cout<<"vector:"<<endl;
    print_grid(vector, 1, N);

    cout<<"matrix:"<<endl;
    print_grid(matrix, N, M);

    int *d_vector;
    bytes = N*sizeof(int);
    cudaMalloc(&d_vector, bytes);
    cudaMemcpy(d_vector, vector, bytes, cudaMemcpyHostToDevice);

    int *d_matrix;
    bytes = N*M*sizeof(int);
    cudaMalloc(&d_matrix, bytes);
    cudaMemcpy(d_matrix, matrix, bytes, cudaMemcpyHostToDevice);

    int *result = new int[M]{0};
    int *d_result;
    bytes = M*sizeof(int);
    cudaMalloc(&d_result, bytes);

    int threads = 64;
    int blocks = ceil(float(M)/float(threads));

    vecmat_multiplication<<<blocks,threads>>>(d_vector, d_matrix, d_result, N, M);
    cudaDeviceSynchronize();

    bytes = M*sizeof(int);
    cudaMemcpy(result, d_result, bytes, cudaMemcpyDeviceToHost);

    cout<<"result:"<<endl;
    print_grid(result, 1, M);

    cudaFree(d_vector);
    cudaFree(d_matrix);
    cudaFree(d_result);
    return 0;
}

vector:
46 24 91 22 91 8 80 12 

matrix:
17 53 50 58 
85 17 23 70 
18 49 89 0 
45 81 77 51 
39 60 55 77 
55 8 68 35 
33 93 88 58 
4 72 4 21 

result:
0 0 0 0 




A3. **Parallel Sorting Algorithms-
For Bubble Sort and Merge Sort, based on existing sequential algorithms, design and implement parallel algorithms utilizing all resources available.**

A4. **Parallel Search Algorithm:
Design and implement parallel algorithm utilizing all resources available for
•1. Binary Search for Sorted Array
•2. Best-First Search that (traversal of the graph to reach a target
in the shortest possible path)**